This notebook uses an SGCN module in the bispy package to search the National List SGCN API. It returns and caches the summarized National List records, which include the list of states that have the species in their conservation planning process.

In [1]:
#Import needed packages
import bispy
import json
from IPython.display import display
from joblib import Parallel, delayed

sgcn = bispy.sgcn.Search()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up cache updated ITIS list with species names from ITIS Exploration.ipynb
with open("cache/itis_explore.json", "r") as f:
    itis_explore = json.loads(f.read())

In [3]:
# Use joblib to run multiple requests for SGCN species records in parallel via ITIS scientific names
sgcn_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in itis_explore]) 

In [4]:
#Display result
sgcn_result

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T19:39:57.594248',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Suaeda maritima'},
  'parameters': {'Scientific Name': 'Suaeda maritima', 'Name Source': None},
  'sgcn_species': {'statelist_2005': '',
   'statelist_2015': 'Massachusetts',
   'scientificname': 'Suaeda maritima',
   'commonname': 'herbaceous seepweed',
   'taxonomicgroup': 'Plants',
   'taxonomicrank': 'Species',
   'matchmethod': 'Followed Accepted TSN',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=20662'}},
 {'processing_metadata': {'status': 'failure',
   'date_processed': '2019-08-27T19:39:57.594248',
   'status_message': 'Not Matched',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Elymus elymoides'},
  'parameters': {'Scientific Name': 'Elymus elymoides', 'Name 

In [5]:
# Filter to give just cases where SGCN species names matched with ITIS species names
success_sgcn=[i for i in sgcn_result if i['processing_metadata']['status'] == 'success']
success_sgcn

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T19:39:57.594248',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Suaeda maritima'},
  'parameters': {'Scientific Name': 'Suaeda maritima', 'Name Source': None},
  'sgcn_species': {'statelist_2005': '',
   'statelist_2015': 'Massachusetts',
   'scientificname': 'Suaeda maritima',
   'commonname': 'herbaceous seepweed',
   'taxonomicgroup': 'Plants',
   'taxonomicrank': 'Species',
   'matchmethod': 'Followed Accepted TSN',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=20662'}},
 {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T19:39:57.594248',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Ictidomys tridecemlineatus'},
  'parameters': {'Scientific Name': 'Ictidomys tridec

In [6]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/sgcn.json", success_sgcn))

{'Doc Cache File': 'cache/sgcn.json',
 'Number of Documents in Cache': 98,
 'Document Number 79': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T19:39:57.594248',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Alces alces'},
  'parameters': {'Scientific Name': 'Alces alces', 'Name Source': None},
  'sgcn_species': {'statelist_2005': 'Massachusetts,Michigan,Wisconsin,Wyoming',
   'statelist_2015': 'Vermont,Wyoming',
   'scientificname': 'Alces alces',
   'commonname': 'moose',
   'taxonomicgroup': 'Mammals',
   'taxonomicrank': 'Species',
   'matchmethod': 'Exact Match',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=180703'}}}

Here we look to see which WLCI species names show up on the Wyoming SGCN list

In [ ]:
#Create list of species names that appear on the Wyoming 2005 and/or 2015 SGCN state list along with additional species information
wyoming_sgcn=[]
for rec in [r["sgcn_species"] for r in sgcn_result if r["processing_metadata"]["status"] == "success"]:
    scietific_name=rec['scientificname']
    common_name=rec['commonname']
    list_2005=rec['statelist_2005']
    list_2015=rec['statelist_2005']
    match_method=rec['matchmethod']
    taxonomic_group=rec['taxonomicgroup']
    if "Wyoming" in rec["statelist_2005"] or "Wyoming" in rec["statelist_2015"]:
        wyoming_sgcn.append({'scientificname': scietific_name,'commonname':common_name,'statelist_2005':list_2005, 'statelist_2015':list_2015, 'matchmethod':match_method, 'taxonomicgroup':taxonomic_group })

In [ ]:
#Display list
wyoming_sgcn

15 WLCI species names appear on the Wyoming SGCN 2005 and/or 2015 list. These species names, along with additional information, are cached as a separate list from the SGCN cached document that include WLCI matched species names that appear on the lists of states other than Wyoming. 

In [ ]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/wy_sgcn.json", wyoming_sgcn))

Check to see if any invalid/not accepted ITIS specie names matched with SGCN records

In [ ]:
# Open up the cached updated ITIS list with invalid/not accepted species names from ITIS Exploration.ipynb
with open("cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [ ]:
# Use joblib to run multiple requests for SGCN documents in parallel via the invalid/not accepted ITIS scientific name
invalid_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [ ]:
# Filter to give just cases where SGCN species names matched with invalid/not accepted ITIS names
invalid_result=[i for i in invalid_result if i['processing_metadata']['status'] == 'success']

In [ ]:
len(invalid_result)

No SGCN records macthed with the invalid/not accepted ITIS specie names